In [ ]:
from PyPDF2 import PdfFileReader
import pandas as pd
import numpy as np
import camelot
import tabula

import ctypes
from ctypes.util import find_library
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

In [ ]:
file_path = "ABB-BRSR Report.pdf"
# file_path = "hdfc-report.pdf"
pdf_file = open(file_path,"rb")

In [ ]:
reader = PdfFileReader(pdf_file, strict=False)

num_pages = reader.numPages
print(num_pages)
print(reader.getPage(3))

In [ ]:
## list of Sections to get search pattern and extract tables section wise
#hdfc sections
#sections = ['SECTION A: GENERAL DISCLOSURES','SECTION B: MANAGEMENT AND PROCESS DISCLOSURES','SECTION C: PRINCIPLE-WISE PERFORMANCE DISCLOSURE']
#abb sections
sections = ['GENERAL DISCLOSURES','MANAGEMENT AND PROCESS DISCLOSURES','PRINCIPLE WISE PERFORMANCE DISCLOSURE']

In [ ]:
##Find page number in pdf using search pattern


def fnPDF_FindText(xString, startPage, endPage):
    # xfile : the PDF file in which to look
    # xString : the string to look for
    import re
    PageFound = -1
    #pdfDoc = PdfFileReader(file(xFile, "rb"))
    for i in range(startPage, endPage):
        PgOb = reader.getPage(i)
        Text = PgOb.extractText()
        if re.search(xString,Text):
             print("String - "+xString+", found on Page: "+str(i))
             PageFound = i
             break
    return PageFound

In [ ]:
## Get All Pages for each Section
section_pages = []
# section_pages.append(-1)
start_page=0
for i in range(0,len(sections)):
    start_search_string = sections[i]
    print('Start Search for section '+start_search_string)
    start_page = fnPDF_FindText(start_search_string,start_page,num_pages)
    if start_page != -1:
        section_pages.append(start_page)    
# section_pages.append(num_pages)
if len(section_pages) > 0:
    #remove duplicates
    section_pages = [*set(section_pages)]
    #sort in ascending order
    section_pages.sort()
print(section_pages)

In [ ]:
## test report
test_res = camelot.read_pdf(file_path,pages='1-10')
print(len(test_res))

test_tab_res = tabula.read_pdf(file_path,pages='1-9')
print(len(test_tab_res))



In [ ]:
#companyData = reader.getPage(3).extractText()
#print(companyData)

### Get data usig tables
for i in range(len(section_pages)):
    print('Extract for '+ sections[i])
    endPageNum = section_pages[i]
    if i == len(section_pages)-1:
        endPageNum = num_pages
    else:
        endPageNum = section_pages[i+1]
    section_tables = camelot.read_pdf(file_path,pages=str(section_pages[i])+'-'+str(endPageNum))
    print(len(section_tables))
    output_file =  file_path[:file_path.find('.')] + '_'+ sections[i][: sections[i].find(':')]
    section_tables.export(output_file+'_camelot',f='csv',compress=True)
    tabula.convert_into(file_path, output_file+'_tabula.csv', output_format="csv", pages=list(range(section_pages[i],endPageNum)))

In [ ]:
## use tabula py to extract all Section A tables in csv



## Get Employee Details - Table

In [ ]:
## use camelot
#reads table from pdf file -- need to extract by providing specific areas
employeeDetails = camelot.read_pdf(file_path,pages='3')
# employeeDetails[0].df


## Get CSR Details - Text

In [ ]:
page_6_dt = reader.getPage(6).extractText()
csrdetails_idx=page_6_dt.find('VI. CSR Details')
compliance_idx=page_6_dt.find('VII. Transparency and Disclosures Compliance')
csrDetails = (page_6_dt[csrdetails_idx:compliance_idx])
print(csrDetails)

check_str='Whether CSR is applicable as per section 135 of Companies Act, 2013:'
isCSRApplyIndex = csrDetails.find(check_str)
csrFlag = csrDetails[isCSRApplyIndex+len(check_str)+2:isCSRApplyIndex+len(check_str)+5]
turnover_idx = csrDetails.find('(ii) Turnover: ` ')
turnover_amt_idx = csrDetails.find('crore',turnover_idx)
turnover_amt = csrDetails[turnover_idx+len('(ii) Turnover: ` '):turnover_amt_idx]
print(turnover_amt)
    
if(csrFlag.strip()=='Yes'):
    ## calculate csr amount
    print('Entering csr calculation')
    csr_text= '(iv) Total amount spent on CSR for FY21:  ` '
    csr_idx = csrDetails.find(csr_text)
    csr_amt_idx = csrDetails.find('crore',csr_idx)
    csr_amt = csrDetails[csr_idx+len(csr_text):csr_amt_idx]
    csr_amt = int(csr_amt.replace(',',''))
    print(csr_amt)
    print('Total CSR-Turnover Ratio')
    turnover_amt = int(turnover_amt.replace(',',''))
    print(csr_amt/turnover_amt)
else:
    print('no csr')

### Calulcate Green house gas emissions

In [ ]:
envDetails = camelot.read_pdf(file_path,pages='9')
electrictyConsumption = envDetails[0].df
greenhousegases = envDetails[1].df
print(greenhousegases.head())

In [ ]:
pdf_file.close()